<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">


# Predicting Shots Made Per Game by Kobe Bryant

_Authors: Kiefer Katovich (SF)_

---

In this lab you'll be using regularized regression penalties — ridge, lasso, and elastic net — to try and predict how many shots Kobe Bryant made per game during his career.

The Kobe Shots data set contains hundreds of columns representing different characteristics of each basketball game. Fitting an ordinary linear regression using every predictor would dramatically overfit the model, considering the limited number of observations (games) we have available. Plus, many of the predictors have significant multicollinearity. 


**Warning:** Some of these calculations are computationally expensive and may take a while to execute. It may be worthwhile to only use a portion of the data to perform these calculations, especially if you've experienced kernel issues in the past.

---

### 1) Load packages and data.

In [29]:
import numpy as np
import pandas as pd
import patsy

from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [5]:
kobe = pd.read_csv('./datasets/kobe_superwide_games.csv')

---

### 2) Examine the data.

- How many columns are there?
- Examine what the observations (rows) and columns represent.
- Why might regularization be particularly useful for modeling this data?

In [6]:
# A:
kobe

,SHOTS_MADE,AWAY_GAME,SEASON_OPPONENT:atl:1996-97,SEASON_OPPONENT:atl:1997-98,SEASON_OPPONENT:atl:1999-00,SEASON_OPPONENT:atl:2000-01,SEASON_OPPONENT:atl:2001-02,SEASON_OPPONENT:atl:2002-03,SEASON_OPPONENT:atl:2003-04,SEASON_OPPONENT:atl:2004-05,...,ACTION_TYPE:tip_layup_shot,ACTION_TYPE:tip_shot,ACTION_TYPE:turnaround_bank_shot,ACTION_TYPE:turnaround_fadeaway_bank_jump_shot,ACTION_TYPE:turnaround_fadeaway_shot,ACTION_TYPE:turnaround_finger_roll_shot,ACTION_TYPE:turnaround_hook_shot,ACTION_TYPE:turnaround_jump_shot,SEASON_GAME_NUMBER,CAREER_GAME_NUMBER
0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
1,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2
2,2.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3
3,2.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,4
4,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,5


---

### 3) Create predictor and target variables. Standardize the predictors.

Why is normalization necessary for regularized regressions?

Use the `sklearn.preprocessing` class `StandardScaler` to standardize the predictors.

In [20]:
corr = kobe.corrwith(kobe['SHOTS_MADE'])
corr.sort_values(ascending = False)

SHOTS_MADE                                1.000000
SHOT_TYPE:2pt_field_goal                  0.756382
SECONDS_REMAINING                         0.747316
PERIOD                                    0.742676
COMBINED_SHOT_TYPE:jump_shot              0.704276
MINUTES_REMAINING                         0.697127
SHOT_ZONE_AREA:center(c)                  0.612336
SHOT_ZONE_BASIC:mid-range                 0.602949
SHOT_ZONE_RANGE:16-24_ft.                 0.542262
SHOT_ZONE_RANGE:less_than_8_ft.           0.496180
SHOT_ZONE_RANGE:8-16_ft.                  0.491018
SHOT_ZONE_BASIC:restricted_area           0.458061
SHOT_ZONE_AREA:right_side(r)              0.428822
SHOT_ZONE_BASIC:in_the_paint_(non-ra)     0.412031
SHOT_ZONE_AREA:right_side_center(rc)      0.406553
COMBINED_SHOT_TYPE:layup                  0.387242
SHOT_ZONE_AREA:left_side(l)               0.379637
SHOT_ZONE_AREA:left_side_center(lc)       0.374991
SHOT_ZONE_RANGE:24+_ft.                   0.313042
SHOT_ZONE_BASIC:above_the_break

In [23]:
features = ['SHOT_TYPE:2pt_field_goal','SECONDS_REMAINING',
            'PERIOD','COMBINED_SHOT_TYPE:jump_shot',
            'MINUTES_REMAINING','SHOT_ZONE_AREA:center(c)','SHOT_ZONE_BASIC:mid-range',
           'SHOT_ZONE_RANGE:16-24_ft.','SHOT_ZONE_RANGE:less_than_8_ft.','SHOT_ZONE_RANGE:8-16_ft.',
           'SHOT_ZONE_BASIC:restricted_area','SHOT_ZONE_AREA:right_side(r)','SHOT_ZONE_BASIC:in_the_paint_(non-ra)',
           'COMBINED_SHOT_TYPE:layup']
X = kobe[features]

In [27]:
# A:

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

ss = StandardScaler()

X_scaled = ss.fit_transform(X)

y = kobe['SHOTS_MADE']

X_train,X_test, y_train, y_test = train_test_split(X_scaled,y, random_state = 42)

---

### 4. Build a linear regression predicting `SHOTS_MADE` from the rest of the columns.

Cross-validate the $R^2$ of an ordinary linear regression model with 10 cross-validation folds.

How does it perform?

In [33]:
# A:
lr = LinearRegression()
kf = KFold(n_splits=10, shuffle=True, random_state = 42,)

scores = cross_val_score(lr,X_train,y_train, cv = kf)
scores

array([0.48622257, 0.58686895, 0.64621707, 0.66346507, 0.66585719,
       0.65749575, 0.57387573, 0.68573126, 0.67621751, 0.55311813])

---

### 5) Find an optimal value for the ridge regression alpha using `RidgeCV`.

Go to the documentation and [read how RidgeCV works](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).

> *Hint: Once the RidgeCV is fit, the attribute `.alpha_` contains the best alpha parameter it found through cross-validation.*

Recall that ridge performs best when searching alphas through logarithmic space (`np.logspace`). This may take awhile to fit.


In [52]:
# A:

from sklearn.linear_model import Ridge, RidgeCV

r_alphas = np.logspace(0, 5, 200)
ridge_model = RidgeCV(alphas=r_alphas, store_cv_values=True)
ridge_model = ridge_model.fit(X_train,y_train)

optimal_alpha = ridge_model.alpha_

---

### 6) Cross-validate the ridge regression $R^2$ with the optimal alpha.

Is it better than the linear regression? If so, why might this be?

In [54]:
# A:
Ridge(alpha = optimal_alpha)



Ridge(alpha=4.768611697714471, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

---

### 7) Find an optimal value for lasso regression alpha using `LassoCV`.

Go to the documentation and [read how LassoCV works](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html). It is very similar to `RidgeCV`.

> *Hint: Again, once the `LassoCV` is fit, the attribute `.alpha_` contains the best alpha parameter it found through cross-validation.*

Recall that lasso, unlike ridge, performs best when searching for alpha through linear space (`np.linspace`). However, you can actually let the LassoCV decide what alphas to use itself by setting the keyword argument `n_alphas=` to however many alphas you want it to search over. We recommend letting scikit-learn choose the range of alphas.

_**Tip:** If you find your CV taking a long time and you're not sure if it's working, set `verbose =1`._

In [68]:
# A:
from sklearn.linear_model import Lasso, LassoCV

l_alphas = np.arange(0.001, 0.15, 0.0025)
lasso_model = LassoCV(alphas=l_alphas)
lasso_model = lasso_model.fit(X_train,y_train)

optimal_alpha_l = lasso_model.alpha_

C:\Users\tbaca\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


---

### 8) Cross-validate the lasso $R^2$ with the optimal alpha.

Is it better than the linear regression? Is it better than ridge? What do the differences in results imply about the issues with the data set?

In [69]:
# A:
Lasso(alpha = optimal_alpha_l)

Lasso(alpha=0.013500000000000002, copy_X=True, fit_intercept=True,
   max_iter=1000, normalize=False, positive=False, precompute=False,
   random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

---

### 9) Look at the coefficients for variables in the lasso.

1. Show the coefficient for variables, ordered from largest to smallest coefficient by absolute value.
2. What percent of the variables in the original data set are "zeroed-out" by the lasso?
3. What are the most important predictors for how many shots Kobe made in a game?

> **Note:** If you only fit the lasso within `cross_val_score`, you'll have to refit it outside of that function to pull out the coefficients.

In [70]:
# A:

lasso_model.coef_

array([ 0.        ,  0.23026127,  0.        ,  1.51393858, -0.        ,
        0.13588207, -0.        ,  0.43171698, -0.        ,  0.21857255,
        1.55225313, -0.02208988,  0.07682286, -0.50803186])

## ---

### 10) Find an optimal value for elastic net regression alpha using `ElasticNetCV`.

Go to the documentation and [read how ElasticNetCV works](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNetCV.html).

Note that here you'll be optimizing both the alpha parameter and the l1_ratio:
- `alpha`: Strength of regularization.
- `l1_ratio`: Amount of ridge vs. lasso (0 = all ridge, 1 = all lasso).
    
Do not include 0 in the search for `l1_ratio` — it won't allow it and will break.

You can use `n_alphas` for the alpha parameters instead of setting your own values, which we highly recommend.

Also, be careful setting too many l1_ratios over cross-validation folds in your search. It can take a long time if you choose too many combinations and, for the most part, there are diminishing returns in this data.

In [74]:
# A:
from sklearn.linear_model import ElasticNet, ElasticNetCV

enet_alphas = np.arange(0.5, 1.0, 0.005)
enet_ratio = 0.5
enet_model = ElasticNetCV(alphas=enet_alphas, l1_ratio=enet_ratio, cv=5)
enet_model = enet_model.fit(X_train, y_train)

---

### 11) Cross-validate the elastic net $R^2$ with the optimal alpha and l1_ratio.

How does it compare to the ridge and lasso regularized regressions?

In [75]:
# A:


0.5

---

### 12. [Bonus] Compare the residuals for ridge and lasso visually.


In [ ]:
# A: Maybe a jointplot?